In [8]:
import pandas as pd
import numpy as np

# Load datasets
train = pd.read_csv(r"C:\Users\hafiz\OneDrive\Documents\Visual Studio Code 2025\walmart_data\train.csv", parse_dates=['Date'])
features = pd.read_csv(r"C:\Users\hafiz\OneDrive\Documents\Visual Studio Code 2025\walmart_data\features.csv", parse_dates=['Date'])
test = pd.read_csv(r"C:\Users\hafiz\OneDrive\Documents\Visual Studio Code 2025\walmart_data\test.csv")
stores = pd.read_csv(r"C:\Users\hafiz\OneDrive\Documents\Visual Studio Code 2025\walmart_data\stores.csv")
sample_submission = pd.read_csv(r"C:\Users\hafiz\OneDrive\Documents\Visual Studio Code 2025\walmart_data\sampleSubmission.csv")

# Check shapes
print("Train:", train.shape)
print("Features:", features.shape)
print("Test:", test.shape)
print("Stores:", stores.shape)
print("SampleSubmission:", sample_submission.shape)

# Merge features into train (Store + Date)
df = pd.merge(train, features, on=['Store','Date'], how='left')

# Merge stores info
df = pd.merge(df, stores, on='Store', how='left')

# Sort by Store, Dept, Date (for consistency)
df.sort_values(['Store','Dept','Date'], inplace=True)
df.reset_index(drop=True, inplace=True)

print("Merged dataset shape:", df.shape)
df.head()

# Drop duplicate IsHoliday_x and rename IsHoliday_y
df.drop('IsHoliday_x', axis=1, inplace=True)
df.rename(columns={'IsHoliday_y': 'IsHoliday'}, inplace=True)

# Check Missing Values
print("Missing values per column:\n", df.isnull().sum())

# Handle missing values
# Fill NaN in markdowns with 0 (no promotion that week)
for col in ['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# Fill NaN in CPI & Unemployment with median
if 'CPI' in df.columns:
    df['CPI'] = df['CPI'].fillna(df['CPI'].median())
if 'Unemployment' in df.columns:
    df['Unemployment'] = df['Unemployment'].fillna(df['Unemployment'].median())

# Save file as Fact Table
output_file = r"C:\Users\hafiz\OneDrive\Documents\Visual Studio Code 2025\walmart_data\FactSales.csv"
df.to_csv(output_file, index=False)
print(f"FactSales table saved as: {output_file}")

Train: (421570, 5)
Features: (8190, 12)
Test: (115064, 4)
Stores: (45, 3)
SampleSubmission: (115064, 2)
Merged dataset shape: (421570, 17)
Missing values per column:
 Store                0
Dept                 0
Date                 0
Weekly_Sales         0
Temperature          0
Fuel_Price           0
MarkDown1       270889
MarkDown2       310322
MarkDown3       284479
MarkDown4       286603
MarkDown5       270138
CPI                  0
Unemployment         0
IsHoliday            0
Type                 0
Size                 0
dtype: int64
FactSales table saved as: C:\Users\hafiz\OneDrive\Documents\Visual Studio Code 2025\walmart_data\FactSales.csv


In [ ]:
import pandas as pd

# Reload data if running separately
df = pd.read_csv(r"C:\Users\hafiz\OneDrive\Documents\Visual Studio Code 2025\walmart_data\FactSales.csv")
stores = pd.read_csv(r"C:\Users\hafiz\OneDrive\Documents\Visual Studio Code 2025\walmart_data\stores.csv")

# ---- DimStore ----
dim_store = stores.copy()
dim_store.to_csv(r"C:\Users\hafiz\OneDrive\Documents\Visual Studio Code 2025\walmart_data\DimStore.csv", index=False)
print("DimStore saved.")

# ---- DimDept ----
dim_dept = df[['Dept']].drop_duplicates().reset_index(drop=True)
dim_dept['DeptKey'] = dim_dept.index + 1
dim_dept.to_csv(r"C:\Users\hafiz\OneDrive\Documents\Visual Studio Code 2025\walmart_data\DimDept.csv", index=False)
print("DimDept saved.")

# ---- DimDate ----
dim_date = pd.DataFrame({
    'Date': pd.date_range(df['Date'].min(), df['Date'].max())
})
dim_date['Year'] = dim_date['Date'].dt.year
dim_date['Month'] = dim_date['Date'].dt.month
dim_date['Week'] = dim_date['Date'].dt.isocalendar().week
dim_date['Quarter'] = dim_date['Date'].dt.quarter
dim_date['Day'] = dim_date['Date'].dt.day
dim_date['IsHoliday'] = dim_date['Date'].isin(df[df['IsHoliday']==True]['Date']).astype(int)

dim_date.to_csv(r"C:\Users\hafiz\OneDrive\Documents\Visual Studio Code 2025\walmart_data\DimDate.csv", index=False)
print("DimDate saved.")